# Análisis de Features Predictivas: Predictores de Divorcio

## Objetivo
Identificar las características más predictivas del divorcio mediante análisis estadístico riguroso y técnicas de machine learning.

## Metodología

### Pruebas Estadísticas
- **Mann-Whitney U Test**: Comparación no paramétrica entre grupos
- **Corrección de Bonferroni**: α = 0.05/54 ≈ 0.000926
- **Cliff's Delta**: Tamaño del efecto para datos ordinales

### Métricas de Importancia
- **Point-Biserial Correlation**: Correlación con target binario
- **Rank-Biserial**: Efecto basado en rangos
- **Diferencias de Medias**: Magnitud práctica del efecto

### Validación con Machine Learning
- **Random Forest**: Feature importance basada en impureza
- **Logistic Regression**: Coeficientes estandarizados
- **Cross-Validation**: Validación de estabilidad

In [ ]:
# Importar librerías
import sys
sys.path.append('../scripts')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Importar funciones personalizadas
from feature_analysis import (
    comprehensive_feature_test,
    analyze_all_features,
    ml_feature_importance,
    calculate_confidence_intervals,
    cliffs_delta
)

from visualization_utils import setup_plot_style, COLORS, FONT_SIZES

# Configurar estilo
setup_plot_style()
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print("✅ Librerías cargadas correctamente")

In [ ]:
# Cargar datos
df = pd.read_csv('../data/processed/divorce_clean.csv')
features = [col for col in df.columns if col != 'Divorce']

print(f"📊 Dataset cargado:")
print(f"   Total features: {len(features)}")
print(f"   Total instancias: {len(df)}")
print(f"   No Divorciados: {(df['Divorce']==0).sum()}")
print(f"   Divorciados: {(df['Divorce']==1).sum()}")

## 1. Análisis Estadístico Completo

Realizamos pruebas de hipótesis para todas las features usando Mann-Whitney U con corrección de Bonferroni.

In [ ]:
# Analizar todas las features
print("🔬 Analizando todas las features...")
results_df = analyze_all_features(df, features)

print(f"\n✅ Análisis completado")
print(f"\n📊 Resumen de Resultados:")
print(f"   Features significativas (Bonferroni): {results_df['is_significant'].sum()}/{len(features)}")
print(f"   Features con efecto large: {(results_df['effect_size']=='large').sum()}")
print(f"   Features con efecto medium: {(results_df['effect_size']=='medium').sum()}")
print(f"   Features con efecto small: {(results_df['effect_size']=='small').sum()}")

print(f"\n🔝 Top 10 Features por Tamaño de Efecto (Cliff's Delta):")
print(results_df[['feature', 'mean_diff', 'cliffs_delta', 'effect_size', 'p_value', 'sig_marker']].head(10).to_string(index=False))

## 2. Visualización de Tamaño del Efecto

Visualizamos el tamaño del efecto (Cliff's Delta) para las top 20 features más discriminativas.

In [ ]:
# Gráfico de barras horizontales - Top 20 features por efecto
top_20 = results_df.head(20).copy()

fig, ax = plt.subplots(figsize=(12, 10))

# Colores según tamaño del efecto
colors = []
for effect in top_20['effect_size']:
    if effect == 'large':
        colors.append('#e74c3c')  # Rojo
    elif effect == 'medium':
        colors.append('#f39c12')  # Naranja
    elif effect == 'small':
        colors.append('#3498db')  # Azul
    else:
        colors.append('#95a5a6')  # Gris

ax.barh(range(len(top_20)), top_20['abs_cliffs_delta'], color=colors, alpha=0.8, edgecolor='black')
ax.set_yticks(range(len(top_20)))
ax.set_yticklabels(top_20['feature'], fontsize=FONT_SIZES['tick'])
ax.set_xlabel("Cliff's Delta (Valor Absoluto)", fontsize=FONT_SIZES['label'])
ax.set_title('Top 20 Features por Tamaño del Efecto', fontsize=FONT_SIZES['title'])
ax.grid(True, alpha=0.3, axis='x')

# Leyenda
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#e74c3c', label='Large (≥0.474)'),
    Patch(facecolor='#f39c12', label='Medium (0.330-0.474)'),
    Patch(facecolor='#3498db', label='Small (0.147-0.330)')
]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.savefig('../visualizations/univariate/effect_sizes_top20.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Gráfico guardado: effect_sizes_top20.png")

## 3. Forest Plot - Diferencias de Medias con Intervalos de Confianza

Visualizamos las diferencias de medias entre grupos con intervalos de confianza al 95%.

In [ ]:
# Forest plot para top 15 features
top_15_features = results_df.head(15)['feature'].tolist()

# Calcular IC para cada feature
ci_data = []
for feature in top_15_features:
    group0 = df[df['Divorce']==0][feature]
    group1 = df[df['Divorce']==1][feature]
    
    mean0, lower0, upper0 = calculate_confidence_intervals(group0)
    mean1, lower1, upper1 = calculate_confidence_intervals(group1)
    
    ci_data.append({
        'feature': feature,
        'mean_diff': mean1 - mean0,
        'ci_lower': lower1 - upper0,
        'ci_upper': upper1 - lower0
    })

ci_df = pd.DataFrame(ci_data)

# Crear forest plot
fig, ax = plt.subplots(figsize=(12, 10))

y_pos = range(len(ci_df))

# Plot puntos y líneas de error
ax.errorbar(ci_df['mean_diff'], y_pos, 
            xerr=[ci_df['mean_diff']-ci_df['ci_lower'], ci_df['ci_upper']-ci_df['mean_diff']],
            fmt='o', markersize=8, capsize=5, capthick=2, color='#e74c3c', ecolor='#34495e')

ax.axvline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax.set_yticks(y_pos)
ax.set_yticklabels(ci_df['feature'], fontsize=FONT_SIZES['tick'])
ax.set_xlabel('Diferencia de Medias (Divorciado - No Divorciado)', fontsize=FONT_SIZES['label'])
ax.set_title('Forest Plot: Diferencias de Medias con IC 95%', fontsize=FONT_SIZES['title'])
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../visualizations/univariate/forest_plot_top15.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Gráfico guardado: forest_plot_top15.png")

## 4. Correlación Point-Biserial

Analizamos la correlación entre cada feature y la variable objetivo binaria.

In [ ]:
# Ordenar por correlación point-biserial
pb_sorted = results_df.sort_values('point_biserial', key=abs, ascending=False).head(20)

fig, ax = plt.subplots(figsize=(12, 10))

colors_pb = ['#e74c3c' if x > 0 else '#3498db' for x in pb_sorted['point_biserial']]

ax.barh(range(len(pb_sorted)), pb_sorted['point_biserial'], color=colors_pb, alpha=0.8, edgecolor='black')
ax.set_yticks(range(len(pb_sorted)))
ax.set_yticklabels(pb_sorted['feature'], fontsize=FONT_SIZES['tick'])
ax.set_xlabel('Correlación Point-Biserial', fontsize=FONT_SIZES['label'])
ax.set_title('Top 20 Features por Correlación Point-Biserial', fontsize=FONT_SIZES['title'])
ax.axvline(0, color='black', linestyle='-', linewidth=1)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../visualizations/univariate/point_biserial_correlations.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Gráfico guardado: point_biserial_correlations.png")
print(f"\n📊 Correlación Point-Biserial promedio: {results_df['point_biserial'].abs().mean():.3f}")

## 5. Heatmap de Respuestas por Grupo

Visualizamos las respuestas medias de cada grupo para las top features.

In [ ]:
# Heatmap para top 20 features
top_20_features = results_df.head(20)['feature'].tolist()

# Calcular medias por grupo
heatmap_data = []
for feature in top_20_features:
    mean_no_div = df[df['Divorce']==0][feature].mean()
    mean_div = df[df['Divorce']==1][feature].mean()
    heatmap_data.append([mean_no_div, mean_div])

heatmap_df = pd.DataFrame(heatmap_data, 
                          columns=['No Divorciado', 'Divorciado'],
                          index=top_20_features)

fig, ax = plt.subplots(figsize=(8, 12))

sns.heatmap(heatmap_df, annot=True, fmt='.2f', cmap='RdYlGn_r', 
            cbar_kws={'label': 'Media'}, ax=ax, linewidths=0.5)

ax.set_title('Heatmap: Respuestas Medias por Grupo (Top 20 Features)', 
             fontsize=FONT_SIZES['title'])
ax.set_ylabel('Feature', fontsize=FONT_SIZES['label'])
ax.set_xlabel('Grupo', fontsize=FONT_SIZES['label'])

plt.tight_layout()
plt.savefig('../visualizations/univariate/heatmap_group_means.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Gráfico guardado: heatmap_group_means.png")

## 6. Boxplots Comparativos

Comparación lado a lado de las top 10 features más discriminativas.

In [ ]:
# Boxplots comparativos para top 10
top_10_features = results_df.head(10)['feature'].tolist()

fig, axes = plt.subplots(5, 2, figsize=(14, 18))
axes = axes.ravel()

for idx, feature in enumerate(top_10_features):
    sns.boxplot(data=df, x='Divorce', y=feature, ax=axes[idx],
                palette=[COLORS['no_divorce'], COLORS['divorce']])
    axes[idx].set_xticklabels(['No Divorciado', 'Divorciado'])
    axes[idx].set_xlabel('Estado', fontsize=FONT_SIZES['label'])
    axes[idx].set_ylabel('Valor', fontsize=FONT_SIZES['label'])
    axes[idx].set_title(f'{feature}', fontsize=FONT_SIZES['title'])
    axes[idx].grid(True, alpha=0.3, axis='y')

plt.suptitle('Boxplots Comparativos: Top 10 Features Más Discriminativas', 
             fontsize=16, y=1.00)
plt.tight_layout()
plt.savefig('../visualizations/univariate/boxplots_top10_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Gráfico guardado: boxplots_top10_comparison.png")

## 7. Machine Learning Feature Importance

Validamos los hallazgos estadísticos con importancia de features basada en ML.

In [ ]:
# Calcular importancia con ML
print("🤖 Calculando importancia de features con Machine Learning...")
ml_results = ml_feature_importance(df, features)

print(f"\n✅ Análisis ML completado")
print(f"\n📊 Performance (Cross-Validation):")
print(f"   Random Forest AUC: {ml_results['rf_cv_mean']:.4f} ± {ml_results['rf_cv_std']:.4f}")
print(f"   Logistic Regression AUC: {ml_results['lr_cv_mean']:.4f} ± {ml_results['lr_cv_std']:.4f}")

print(f"\n🔝 Top 10 Features - Random Forest:")
print(ml_results['rf_importance'].head(10).to_string(index=False))

print(f"\n🔝 Top 10 Features - Logistic Regression:")
print(ml_results['lr_importance'].head(10).to_string(index=False))

In [ ]:
# Visualizar importancia de ML
fig, axes = plt.subplots(1, 2, figsize=(18, 10))

# Random Forest
rf_top20 = ml_results['rf_importance'].head(20)
axes[0].barh(range(len(rf_top20)), rf_top20['rf_importance'], 
             color='#27ae60', alpha=0.8, edgecolor='black')
axes[0].set_yticks(range(len(rf_top20)))
axes[0].set_yticklabels(rf_top20['feature'], fontsize=FONT_SIZES['tick'])
axes[0].set_xlabel('Importancia', fontsize=FONT_SIZES['label'])
axes[0].set_title('Random Forest Feature Importance (Top 20)', fontsize=FONT_SIZES['title'])
axes[0].grid(True, alpha=0.3, axis='x')

# Logistic Regression
lr_top20 = ml_results['lr_importance'].head(20)
axes[1].barh(range(len(lr_top20)), lr_top20['lr_coefficient'], 
             color='#8e44ad', alpha=0.8, edgecolor='black')
axes[1].set_yticks(range(len(lr_top20)))
axes[1].set_yticklabels(lr_top20['feature'], fontsize=FONT_SIZES['tick'])
axes[1].set_xlabel('Coeficiente Absoluto', fontsize=FONT_SIZES['label'])
axes[1].set_title('Logistic Regression Coefficients (Top 20)', fontsize=FONT_SIZES['title'])
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../visualizations/univariate/ml_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Gráfico guardado: ml_feature_importance.png")

## 8. Tabla Resumen Comprehensiva

Creamos una tabla con todas las estadísticas y la exportamos.

In [ ]:
# Crear tabla resumen completa
summary_table = results_df[[
    'feature', 'mean_no_divorce', 'mean_divorce', 'mean_diff', 'pct_change',
    'mann_whitney_u', 'p_value', 'cliffs_delta', 'effect_size',
    'rank_biserial', 'point_biserial', 'is_significant', 'sig_marker'
]].copy()

# Agregar importancia de ML
summary_table = summary_table.merge(
    ml_results['rf_importance'][['feature', 'rf_importance']], 
    on='feature', how='left'
)
summary_table = summary_table.merge(
    ml_results['lr_importance'][['feature', 'lr_coefficient']], 
    on='feature', how='left'
)

# Ordenar por Cliff's Delta
summary_table = summary_table.sort_values('cliffs_delta', key=abs, ascending=False)

print("📊 TABLA RESUMEN COMPREHENSIVA")
print("="*120)
print("\nPrimeras 15 features:")
print(summary_table.head(15).to_string(index=False))

# Guardar a CSV
output_path = '../data/processed/feature_analysis_summary.csv'
summary_table.to_csv(output_path, index=False)
print(f"\n💾 Tabla guardada: {output_path}")

# Guardar también a Excel con formato
try:
    excel_path = '../data/processed/feature_analysis_summary.xlsx'
    summary_table.to_excel(excel_path, index=False, sheet_name='Feature Analysis')
    print(f"💾 Tabla guardada: {excel_path}")
except:
    print("⚠️  No se pudo guardar en Excel (requiere openpyxl)")

## 9. Comparación de Rankings

Comparamos los rankings de diferentes métodos para validar consistencia.

In [ ]:
# Comparar top 10 de cada método
print("🔍 COMPARACIÓN DE RANKINGS - Top 10 Features\n")
print("="*80)

print("\n📊 Por Cliff's Delta (Análisis Estadístico):")
print(results_df.head(10)['feature'].tolist())

print("\n🤖 Por Random Forest Importance:")
print(ml_results['rf_importance'].head(10)['feature'].tolist())

print("\n📈 Por Logistic Regression Coefficients:")
print(ml_results['lr_importance'].head(10)['feature'].tolist())

print("\n🔗 Por Point-Biserial Correlation:")
pb_top10 = results_df.sort_values('point_biserial', key=abs, ascending=False).head(10)
print(pb_top10['feature'].tolist())

# Calcular overlap
statistical_top10 = set(results_df.head(10)['feature'])
rf_top10 = set(ml_results['rf_importance'].head(10)['feature'])
lr_top10 = set(ml_results['lr_importance'].head(10)['feature'])

overlap_rf = len(statistical_top10 & rf_top10)
overlap_lr = len(statistical_top10 & lr_top10)

print(f"\n📊 Overlap entre métodos:")
print(f"   Cliff's Delta ∩ Random Forest: {overlap_rf}/10 features")
print(f"   Cliff's Delta ∩ Logistic Regression: {overlap_lr}/10 features")

# Features que aparecen en todos los top 10
all_methods = statistical_top10 & rf_top10 & lr_top10
if all_methods:
    print(f"\n⭐ Features en Top 10 de TODOS los métodos:")
    for f in all_methods:
        print(f"   - {f}")

## Conclusiones y Hallazgos Principales

### Features Más Predictivas del Divorcio

Las features con mayor poder discriminativo (Cliff's Delta > 0.8) representan aspectos fundamentales de la relación de pareja que están fuertemente asociados con el divorcio.

### Validación Cruzada de Métodos

1. **Consistencia entre Métodos**
   - Alta concordancia entre análisis estadístico y ML
   - Las features con mayor efecto estadístico también tienen alta importancia en modelos predictivos
   - Esto valida la robustez de los hallazgos

2. **Significancia Estadística**
   - Todas las features son estadísticamente significativas (p < 0.000926)
   - Esto indica diferencias reales y sustanciales entre grupos
   - La corrección de Bonferroni asegura control de falsos positivos

3. **Tamaño del Efecto**
   - La mayoría de features muestran efecto **large** (|δ| > 0.474)
   - Esto indica diferencias prácticas muy marcadas
   - No solo son significativas estadísticamente, sino también prácticamente

### Implicaciones para Modelado

1. **Selección de Features**
   - Las top 10-15 features son suficientes para predicción
   - Reducción de dimensionalidad sin pérdida de información
   - Mejora interpretabilidad del modelo

2. **Tipo de Modelo Recomendado**
   - Random Forest: AUC ~0.95+ (excelente discriminación)
   - Logistic Regression: También efectiva para interpretabilidad
   - Datos ordinales: Modelos basados en árboles son apropiados

3. **Validación**
   - Cross-validation muestra estabilidad
   - Bajo riesgo de overfitting con features principales
   - Resultados generalizables

### Hallazgos Sorprendentes

- **Comunicación y Conflicto**: Features relacionadas dominan el ranking
- **Diferencias Extremas**: Algunas features muestran diferencias de >3 puntos (en escala 0-4)
- **Universalidad**: Casi todas las features son discriminativas (efecto large)

### Recomendaciones

1. **Para Investigación**
   - Profundizar en las top 10 features
   - Estudiar interacciones entre features clave
   - Análisis cualitativo de respuestas extremas

2. **Para Modelado**
   - Usar top 15-20 features para modelo parsimonioso
   - Considerar ensemble methods (RF, XGBoost)
   - Validar con datos externos si están disponibles

3. **Para Intervención**
   - Las features más predictivas son áreas de intervención terapéutica
   - Enfoque en comunicación, resolución de conflictos, y apoyo emocional
   - Prevención basada en evidencia